# M07(VL06) Daiyun Dong

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import requests
from pprint import pprint

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, KFold, \
    GroupKFold
from scipy.stats import pearsonr, mode
import matplotlib.pyplot as plt
import seaborn as sns

# Import data from protein sequence embedding repository.
sys.path.append("/Users/dongdaiyun/Documents/Python/ProteinBioinfo/protein-sequence-embedding-iclr2019")
from src.alphabets import Uniprot21
from torch.nn.utils.rnn import PackedSequence
from src.utils import pack_sequences, unpack_sequences

import torch
%matplotlib inline


In [ ]:
# Load stability data
# Update this path based upon where you downloaded data.csv

stability_data = pd.read_csv("/Users/dongdaiyun/Documents/Python/ProteinBioinfo/pythonProject/data.csv")
stability_data.head()

In [ ]:
def mutate_sequence(row): 
    if not np.isnan(row.position):
        pos = int(row.position) - 1
        assert pos < len(row.sequence)
        new_sequence = list(row.sequence)
        new_sequence[pos] = row.mutation
        new_sequence = "".join(new_sequence)
        if row.sequence[pos] == row.wild_type:
            return new_sequence
        else:
            return row.sequence
stability_data.loc[:, "sequence_mut"] = stability_data.apply(mutate_sequence, axis=1)

In [ ]:
# Add simple features
stability_data.head()
stability_data = stability_data[~stability_data['protein_name'].isnull()]
stability_data = stability_data[~stability_data['ddG'].isnull()]
prediction_cols = ['pdb_id', 'wild_type', 'mutation']

# Convert categorical variables into simple One-hot encoding
X = pd.get_dummies(stability_data.loc[:, prediction_cols])

# ddG or ΔΔG is the target variable in this case
y = stability_data.ddG
predictions, truths = [], []
groups = stability_data.protein_name

for seed in [0, 1, 2]:
    for train_index, test_index in GroupKFold(7).split(X, y, groups):
        X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        reg = MLPRegressor(50, random_state=seed, max_iter=2000)
        reg.fit(X_train, y_train)
        predictions += reg.predict(X_test).tolist()
        truths += y_test.tolist()

print("Test correlation: ", pearsonr(truths, predictions)[0])

The output:
Test correlation:  0.1204051159467796

## Discussion
### What criteria did you use to evaluate the performance of the model? What changes (if any) could be made to optimize the evaluation?

The performance of the model was evaluated using the Pearson correlation coefficient, which measures the linear correlation between the model-predicted ΔΔG values and the experimentally determined ΔΔG values. The range of the Pearson correlation coefficient is from -1 to 1, with its absolute value closer to 1 indicating stronger predictive capability of the model and higher consistency between the predicted results and actual situations.

Additionally, the Mean Squared Error (MSE) could also be used to assess the consistency between the model's predictions and actual situations. The MSE calculates the average of the squares of the differences between the model's predictions and the actual values. A smaller MSE indicates higher accuracy of the model's predictions.





### How do you interpret the Pearson correlation scores that you generated? What conclusions about the structure and function of your generated mutants can you make based on this analysis?

The obtained Pearson correlation coefficient of 0.1204, whose absolute value is not zero but relatively small, indicates a very slight positive linear relationship between the predicted ΔΔG values and the actual ΔΔG values. This suggests that the model built with our input data is of poor quality, with low accuracy in predicting the thermal stability of protein variants.

In this model, three features (secondary structure, wild-type amino acid, and mutation type) were used as independent variables to input into the model, with the thermal stability change (ΔΔG) of the protein variants as the dependent variable that the model aims to predict. Given the low accuracy indicated by the Pearson correlation coefficient, it suggests that the mutations might not typically impact the overall structure and function of the proteins, occurring in non-critical areas of the proteins, which leads to the training being unable to predict changes in thermal stability accurately.




### What are the steps used for thermal stability predictions using this approach? Are there any assumptions that limit the broad applicability of the approach?

The process begins with preparing protein sequence and structure data, followed by generating mutant sequences. Then, features for training the model are extracted, the variables to be predicted are defined, and the model is trained and predicted using machine learning. Finally, the model's performance is assessed using statistical methods.

The model may assume that the same mutation on the same secondary structure has the same impact on thermal stability, ignoring the possibility that specific regions or types of mutations may have different impacts on thermal stability. In reality, structural and functional differences between proteins may cause the same mutation to have different effects in different proteins. For example, a single amino acid change in the structural core of a protein may lead to significant stability loss; whereas mutations in non-critical areas might not significantly affect the protein's structure and function. Additionally, protein stability is also related to tertiary structure, and incorporating the tertiary structure of proteins into the model training could potentially improve the accuracy of predictions.



### Are there sequence characteristics or limitations that would cause this analysis to fail? Can you think of conditions that would result in tremendously inaccurate stability calculations? Why or why not?

When encountering amino acid mutations in special regions of proteins, these mutations may not follow the pattern of most mutations leading to ΔΔG changes, such as mutations on the protein surface having minor impacts on the overall structure and function of the protein. Moreover, specific amino acid mutations may have synergistic effects, and machine learning models built during the training process may not capture this synergistic effect well, leading to prediction failures.
